# ResNet18 vs ViT-b/16 — Colab Trainer

This notebook mounts Google Drive, clones your GitHub repo, installs dependencies,
links your **data/** folder from Drive, and runs your shell scripts:

- `scripts/run_resnet.sh`
- `scripts/run_vit.sh`
- (optional) `scripts/eval.sh`

All logs and outputs are copied back to Drive.

In [ ]:
# Check GPU (optional)
!nvidia-smi || echo "No GPU detected"

In [ ]:
#@title 🔌 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title ⚙️ Configuration { run: "auto" }
#@markdown Fill these and run the cells below.

REPO_URL = "https://github.com/Alejandro-Fuste/AI-ResNet-vs-ViT" #@param {type:"string"}
BRANCH = "main"                                                #@param {type:"string"}
PROJECT_NAME = "AI-RESNET-vs-ViT"                              #@param {type:"string"}
DRIVE_DATA = "/content/drive/MyDrive/UCF/Courses/Fall2025/ComputerVisionSystems/HW_1/data.zip"                     #@param {type:"string"}
DRIVE_SAVE = "/content/drive/MyDrive/ai-resnet-vit-outputs"    #@param {type:"string"}

RUN_RESNET = True   #@param {type:"boolean"}
RUN_VIT    = True   #@param {type:"boolean"}
ALSO_EVAL  = True   #@param {type:"boolean"}

In [ ]:
# 🧬 Clone or update the repo
import os, subprocess
workdir = "/content"
repo_dir = os.path.join(workdir, PROJECT_NAME)
if os.path.exists(repo_dir):
    print(f"Repo exists at {repo_dir}. Updating…")
    subprocess.run(["git","fetch","--all"], cwd=repo_dir, check=True)
    subprocess.run(["git","checkout", BRANCH], cwd=repo_dir, check=True)
    subprocess.run(["git","pull","--ff-only"], cwd=repo_dir, check=True)
else:
    subprocess.run(["git","clone","-b", BRANCH, "--single-branch", REPO_URL, PROJECT_NAME], cwd=workdir, check=True)
print("Ready:", repo_dir)

## ⚡ Pip-only setup (no Conda) — recommended for Colab

Use this **instead of** the Conda setup if you placed the Colab scripts under `scripts/colab/`.
This will install your `requirements.txt` into the default Colab Python and skip Conda entirely.

In [ ]:
# Run from the repo root: installs requirements via pip
import os, subprocess
repo_dir = os.path.join("/content", PROJECT_NAME)
print("Repo:", repo_dir)
subprocess.run(["bash","-lc", f"cd '{repo_dir}' && bash scripts/colab/setup_env_colab.sh"], check=True)

In [ ]:
# 📦 Install requirements
import os, subprocess, sys
req = os.path.join(repo_dir, "requirements.txt")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "wheel", "setuptools"], check=True)
if os.path.exists(req):
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", req], check=True)
else:
    print("requirements.txt not found:", req)

In [ ]:
# 📂 Unzip data.zip from Drive
import os, zipfile

zip_path = "/content/data.zip"   # <-- change this to match where your zip is
extract_to = "/content/drive/MyDrive/data"     # this will create the 'data/' folder

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extracted to:", extract_to)

In [ ]:
# 🔗 Link Drive data into the repo as ./data
import os, shutil, time
repo_data = os.path.join(repo_dir, "data")
if os.path.exists(repo_data):
    if os.path.islink(repo_data):
        os.unlink(repo_data)
    else:
        backup = repo_data + "_backup_" + time.strftime("%Y%m%d_%H%M%S")
        print("Backing up existing data folder to", backup)
        shutil.move(repo_data, backup)

if os.path.exists(DRIVE_DATA):
    try:
        os.symlink(DRIVE_DATA, repo_data, target_is_directory=True)
        print("Symlinked", DRIVE_DATA, "->", repo_data)
    except Exception as e:
        print("Symlink failed, copying instead…", e)
        shutil.copytree(DRIVE_DATA, repo_data)
else:
    raise SystemExit(f"Drive data path not found: {DRIVE_DATA}")

In [ ]:
# ✅ Make shell scripts executable
import os, stat
scripts = os.path.join(repo_dir, "scripts")
if os.path.isdir(scripts):
    for f in os.listdir(scripts):
        if f.endswith(".sh"):
            p = os.path.join(scripts, f)
            os.chmod(p, 0o775)
            print("chmod +x", p)
else:
    print("No scripts directory found:", scripts)

In [ ]:
# Check GPU (optional)
!nvidia-smi || echo "No GPU detected"

In [ ]:
# 🚀 Run training scripts with logging to Drive
import os, subprocess, sys, time
from pathlib import Path

os.makedirs(DRIVE_SAVE, exist_ok=True)
timestamp = time.strftime("%Y%m%d_%H%M%S")

def tee_log(cmd, log_path, cwd):
    Path(os.path.dirname(log_path)).mkdir(parents=True, exist_ok=True)
    with open(log_path, "w") as f:
        print("[RUN]", " ".join(cmd))
        p = subprocess.Popen(cmd, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            sys.stdout.write(line)
            f.write(line)
        p.wait()
        if p.returncode != 0:
            raise SystemExit(f"Command failed with exit code {p.returncode}")

if RUN_RESNET:
    tee_log(["bash","scripts/colab/run_resnet_colab.sh"], os.path.join(DRIVE_SAVE, f"train_resnet18_{timestamp}.log"), cwd=repo_dir)

if RUN_VIT:
    tee_log(["bash","scripts/colab/run_vit_colab.sh"], os.path.join(DRIVE_SAVE, f"train_vit_b16_{timestamp}.log"), cwd=repo_dir)


In [ ]:
# 📊 Run evaluation (eval.sh) separately
import os, time

timestamp = time.strftime("%Y%m%d_%H%M%S")

if os.path.exists(os.path.join(repo_dir, "scripts", "eval.sh")):
    tee_log(["bash","scripts/colab/eval_colab.sh"], os.path.join(DRIVE_SAVE, f"eval_{timestamp}.log"), cwd=repo_dir)
else:
    print("⚠️ eval.sh not found in scripts/")


In [ ]:
# 💾 Copy repo logs/outputs back into Drive for safekeeping
import os, shutil
repo_logs = os.path.join(repo_dir, "logs")
repo_outs = os.path.join(repo_dir, "outputs")
for src, name in [(repo_logs,"logs"), (repo_outs,"outputs")]:
    if not os.path.exists(src):
        print("Skip (missing):", src); continue
    dst = os.path.join(DRIVE_SAVE, name)
    for root, dirs, files in os.walk(src):
        rel = os.path.relpath(root, src)
        tgt_root = os.path.join(dst, rel) if rel != "." else dst
        os.makedirs(tgt_root, exist_ok=True)
        for file in files:
            s = os.path.join(root, file)
            t = os.path.join(tgt_root, file)
            try:
                shutil.copy2(s, t)
            except Exception as e:
                print("Copy warn:", s, "->", t, e)
print("Synced to", DRIVE_SAVE)

In [ ]:
# 🧾 Freeze environment for reproducibility
import subprocess, sys, time, os
ts = time.strftime("%Y%m%d_%H%M%S")
freeze_path = os.path.join(DRIVE_SAVE, f"pip_freeze_{ts}.txt")
with open(freeze_path, "w") as f:
    subprocess.run([sys.executable, "-m", "pip", "freeze"], text=True, stdout=f)
print("Wrote", freeze_path)

## 🚀 Fast data setup in Colab (upload → unzip locally → link to repo → optional copy back to Drive)

Unzipping **from Google Drive** is slow. This section lets you **upload your dataset ZIP directly into Colab’s `/content`** (local SSD), unzip it quickly, link it into your repo at `./data`, and **optionally copy the extracted folder back to Drive** for reuse.

In [ ]:
# 🔧 Set your paths here
# If you already know your repo folder name under /content, set PROJECT_NAME below.
PROJECT_NAME = "AI-RESNET-VS-VIT"   # <-- change if different
REPO_DIR = f"/content/{PROJECT_NAME}"

# Set where the dataset will be extracted locally in the VM
EXTRACT_DIR = "/content/data"

# Where to save a permanent copy in Drive (optional)
DRIVE_TARGET = "/content/drive/MyDrive/data"

print("Repo dir:", REPO_DIR)

### 1) Upload your ZIP directly into Colab (fastest)
**Option A (drag & drop):** In the left sidebar → **Files** tab → click **Upload** and put your dataset ZIP in `/content/` (root).  
**Option B (helper):** Use the upload widget below and select your ZIP file.

In [ ]:
from google.colab import files
print("Select your ZIP file…")
uploaded = files.upload()  # choose your dataset zip
# Grab the first uploaded filename
ZIP_PATH = None
if uploaded:
    ZIP_PATH = "/content/" + next(iter(uploaded.keys()))
    print("Uploaded:", ZIP_PATH)
else:
    print("No file uploaded. If you used drag-and-drop, set ZIP_PATH manually below.")

In [ ]:
# If you used drag-and-drop into /content, set ZIP_PATH manually like:
# ZIP_PATH = "/content/your_dataset.zip"
try:
    ZIP_PATH
except NameError:
    ZIP_PATH = None

print("ZIP_PATH =", ZIP_PATH)

### 2) Unzip locally in `/content` (much faster than Drive)
This will extract to `EXTRACT_DIR` and skip re-extracting if that folder already exists.

In [ ]:
import os, subprocess

assert os.path.isdir(REPO_DIR), f"Repo not found at: {REPO_DIR}. Set PROJECT_NAME correctly."
assert EXTRACT_DIR.startswith("/content/"), "EXTRACT_DIR should be under /content for speed."

if not ZIP_PATH or not os.path.exists(ZIP_PATH):
    raise SystemExit("ZIP_PATH is not set or file not found. Upload your zip or set ZIP_PATH to its path in /content.")

os.makedirs(EXTRACT_DIR, exist_ok=True)

# Choose tool based on extension; prefer 7z for speed if available
ext = os.path.splitext(ZIP_PATH)[1].lower()
if ext in (".7z", ".xz", ".tar", ".gz", ".tgz", ".bz2"):
    # Install p7zip if needed
    subprocess.run(["bash","-lc","apt-get -qq update && apt-get -qq install -y p7zip-full"], check=True)
    cmd = f"7z x -aoa '{ZIP_PATH}' -o'{EXTRACT_DIR}'"
else:
    # default to unzip for .zip
    cmd = f"unzip -q '{ZIP_PATH}' -d '{EXTRACT_DIR}'"

print("[RUN]", cmd)
subprocess.run(["bash","-lc", cmd], check=True)

# Show a quick tree
subprocess.run(["bash","-lc", f"ls -lah '{EXTRACT_DIR}' | head -n 40"], check=True)

### 3) Link the extracted folder into your repo as `./data`
If a `data` folder already exists in your repo, we back it up and replace it with a symlink to the extracted data.

In [ ]:
import os, shutil, time, pathlib

repo_data = os.path.join(REPO_DIR, "data")

# If repo_data exists, back it up
if os.path.lexists(repo_data):  # includes symlinks
    backup = repo_data + "_backup_" + time.strftime("%Y%m%d_%H%M%S")
    print(f"[INFO] Backing up existing data to: {backup}")
    os.rename(repo_data, backup)

# If EXTRACT_DIR contains a single top-level folder, link to that; else link the dir directly
entries = [e for e in os.listdir(EXTRACT_DIR) if not e.startswith(".")]
if len(entries) == 1 and os.path.isdir(os.path.join(EXTRACT_DIR, entries[0])):
    target = os.path.join(EXTRACT_DIR, entries[0])
else:
    target = EXTRACT_DIR

# Create symlink
os.symlink(target, repo_data, target_is_directory=True)
print("Linked", target, "->", repo_data)

### 4) (Optional) Copy the extracted data back to Google Drive
This persists the unzipped dataset so you don’t have to upload/unzip again next time.

In [ ]:
import os, subprocess

# Ensure Drive is mounted
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Create the target dir and copy (rsync if available, else cp -r)
subprocess.run(["bash","-lc", f"mkdir -p '{DRIVE_TARGET}'"], check=True)

cmd = f"rsync -a --delete '{EXTRACT_DIR}/' '{DRIVE_TARGET}/' || cp -r '{EXTRACT_DIR}'/* '{DRIVE_TARGET}/'"
print("[RUN]", cmd)
subprocess.run(["bash","-lc", cmd], check=True)

print("Copied extracted data to:", DRIVE_TARGET)